In [ ]:
import numpy as np
import os
from greem.utility.config import Rendition, EncodingConfig, EncodingConfigDTO

In [ ]:
cmd = '''
ffmpeg 
-y 
-i ../dataset/ref_265/AncientThought_s000.265 
-i ../dataset/ref_265/AncientThought_s001.265 
-i ../dataset/ref_265/Basketball_s000.265 
-i ../dataset/ref_265/Basketball_s001.265 
-i ../dataset/ref_265/Beauty_s000.265 
-i ../dataset/ref_265/AncientThought_s000.265 

-map 0 -c:v libx264 -preset fast -vf scale=1280:720 output1.mp4 
-map 1 -c:v libx264 -preset slow -vf scale=1280:720 output2.mp4 
-map 2 -c:v libx264 -preset medium -vf scale=1280:720 output3.mp4 
-map 3 -c:v libx264 -preset fast -vf scale=1280:720 output4.mp4 
-map 4 -c:v libx264 -preset fast -vf scale=1280:720 output5.mp4 
-map 5 -c:v libx265 -preset medium -vf scale=1920:1080 output6.mp4 
'''
# -map 0 -c:v libx264 -q:v 1 output1.mp4 
# -map 1 -c:v libx264 -q:v 1 output2.mp4 
# -map 2 -c:v libx264 -q:v 1 output3.mp4 
# -map 3 -c:v libx264 -q:v 1 output4.mp4 
# -map 4 -c:v libx264 -q:v 1 output5.mp4

cmd = cmd.replace('\n', '')
cmd

In [ ]:
def create_multi_video_ffmpeg_command(
    video_input_file_paths: list[str],
    output_directories: list[str],
    dto: EncodingConfigDTO,
    segment_seconds: int = 4,
    cuda_mode: bool = False,
    quiet_mode: bool = False,
    pretty_print: bool = False
) -> str:
    cmd: list[str] = [
        'ffmpeg', '-y', 
    ]
    cmd.extend([f'-i {video}' for video in video_input_file_paths])
    
    bitrate = int(dto.rendition.bitrate)
    fps_repr: str = '' if len(dto.framerate) == 0 else f',fps={dto.framerate}'
    
    for idx in range(len(video_input_file_paths)):
        # map_cmd = f'-map {idx} -c:v mpeg4 -q:v 1 -seg_duration 4 -f dash {output_directories[idx]}/{idx}/video{idx}.mpd'
        # cmd.append(map_cmd)
        map_cmd: list[str] = [
            f'-map {idx}',
            f'-c:v {dto.codec} -preset {dto.preset}', 
            # f'-c:v libx265 -preset {preset}', 
            f'-minrate {bitrate}k -maxrate {bitrate}k -bufsize {3*bitrate}k',
            f'-vf scale={dto.rendition.get_resolution_dir_representation()}',
            fps_repr,
            
            
            # f'-seg_duration {segment_seconds}',
            
            # TODO probably add resolution etc here.
            # get_representation_ffmpeg_flags(renditions, preset, codec),
        ]
        
        cmd.extend(map_cmd)
        
        # cmd.extend(get_representation_ffmpeg_flags(renditions, preset, codec, is_multi_video=False))
        
        cmd.extend([
            f'{output_directories[idx]}output.mp4'
            ])
        
    join_string: str = ' \n' if pretty_print else ' '
    return join_string.join(cmd)


In [ ]:
input_file_paths: list[str] = '''-i ../dataset/ref_265/AncientThought_s000.265 
-i ../dataset/ref_265/AncientThought_s001.265 
-i ../dataset/ref_265/Basketball_s000.265 
-i ../dataset/ref_265/Basketball_s001.265 
-i ../dataset/ref_265/Beauty_s000.265 
-i ../dataset/ref_265/AncientThought_s000.265'''.replace('-i ', '').replace(' ', '').split('\n')
input_file_paths

output_file_paths = [f'{idx}' for idx in range(len(input_file_paths))] * len(input_file_paths)

renditions: list[Rendition] = [Rendition(480, 640, 1), Rendition(1280, 1920, 2)]

presets: list[str] = ['medium', 'fast']

codecs: list[str] = ['h264', 'h265']


In [ ]:
cmd = create_multi_video_ffmpeg_command(
    input_file_paths, output_file_paths,
    renditions[0], presets[0], codecs[0],
    pretty_print=False
)

print(cmd)

In [ ]:
os.system(cmd)